***Note: this is the EcoliPPP.ipynb notebook. The
PDF version "The Escherichia coli Core Model: 
Modular Energetic Bond Graph Analysis of Pentose Phosphate Pathways"
is available [here](EcoliPPPnoPhi.pdf).***

- This is the version without reaction potentials

# Introduction

The Network Thermodynamics/Bond Graph approach of 
<cite data-cite="OstPerKat71,OstPerKat73">Oster, Perlson and Katchalsky (1971,1973)</cite>, extended by <cite data-cite="GawCra14,GawCra16,GawCra17">Gawthrop and Crampin (2014,2016,2017)</cite>,
to modelling biomolecular systems developed independently from the stoichiometric approach 
<cite data-cite="Pal06,Pal11,Pal15"></cite>.

However, the conceptual point of intersection of the two approaches is the fact that the stoichiometric matrix is the modulus of the conceptual multiport transformer linking reactions to species.
This was pointed out by <cite data-cite="CelGre09">Cellier and Greifeneder (2009)</cite>. This means that the two approaches are complementary and each can build on the strengths of the other.

In particular, as discussed here, the Bond Graph approach adds energy to stoichiometry.

This notebook focuses on building modular models of metabolism and consequent pathway analysis based on the Escherichia coli Core Model <cite data-cite="OrtFlePal10">(Orth, Fleming and Palsson,2010)</cite>; in particular, the Glycolysis and Pentose Phosphate portion is extracted and analysed. Following the discussion in the textbook of 
<cite data-cite="GarGri17">Garrett and Grisham (2017)</cite>, section 22.6d, various possible pathways are examined by choosing appropriate chemostats and flowstats.
<cite data-cite="GawCra18">(Gawthrop and Crampin, 2018)</cite>

Assuming steady-state conditions, the corresponding pathway potentials <cite data-cite="Gaw17a">(Gawthrop 2017)</cite> are derived.


## Import some python code
The bond graph analysis uses a number of Python modules:

In [1]:
## Maths library
import numpy as np
import scipy
## BG tools
import BondGraphTools as bgt

## BG stoichiometric utilities
import stoich as st

## Stoichiometric conversion
import CobraExtract as Extract
import stoichBondGraph as stbg

## Potentials
import phiData

## Faraday constant
import scipy.constants as con
F = con.physical_constants['Faraday constant'][0]

## Display
import IPython.display as disp

import copy

## Allow output from within functions
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import importlib as imp

## Units etc
factor = 1
units = ['mV','kJ']

## Control output
quiet = True
computePhi = True
showMu = True

# Extract the model

## Extract full ecoli core model from the CobraPy representation

In [2]:
sm = Extract.extract(cobraname='textbook',Remove=['_C','__' ], 
                     negReaction=['RPI','PGK','PGM','SUCOAS'], quiet=quiet)
print(sm['reaction'])

Extracting stoichiometric matrix from: textbook
Cobra Model name: e_coli_core BondGraphTools name: e_coli_core_abg
Extract.Integer only handles one non-integer per reaction
Multiplying reaction BIOMASS_ECOLIORE ( 12 ) by 0.6684491978609626 to avoid non-integer species 3PG ( 2 )
Multiplying reaction CYTBD ( 15 ) by 2.0 to avoid non-integer species O2 ( 55 )
Multiplying reaction PGK ( 54 ) by -1
Multiplying reaction PGM ( 56 ) by -1
Multiplying reaction RPI ( 65 ) by -1
Multiplying reaction SUCOAS ( 69 ) by -1
['ACALD', 'ACALDT', 'ACKR', 'ACONTA', 'ACONTB', 'ACT2R', 'ADK1', 'AKGDH', 'AKGT2R', 'ALCD2X', 'ATPM', 'ATPS4R', 'BIOMASS_ECOLIORE', 'CO2T', 'CS', 'CYTBD', 'D_LACT2', 'ENO', 'ETOHT2R', 'FBA', 'FBP', 'FORT2', 'FORTI', 'FRD7', 'FRUPTS2', 'FUM', 'FUMT2_2', 'G6PDH2R', 'GAPD', 'GLCPTS', 'GLNS', 'GLNABC', 'GLUDY', 'GLUN', 'GLUSY', 'GLUT2R', 'GND', 'H2OT', 'ICDHYR', 'ICL', 'LDH_D', 'MALS', 'MALT2_2', 'MDH', 'ME1', 'ME2', 'NADH16', 'NADTRHD', 'NH4T', 'O2T', 'PDH', 'PFK', 'PFL', 'PGI', 'PGK'

## Extract Glycolysis, Pentose Phosphate Pathways and TCA (using PDH and PDH)

In [3]:
name = 'GlyPPP_abg'
reaction = []

## Glycolysis
reaction += ['PGI','PFK','FBA','TPI']

## Pentose Phosphate
reaction += ['G6PDH2R','PGL','GND','RPI','TKT2','TALA','TKT1','RPE']

## Create submodel
sGlyPPP = Extract.choose(sm,reaction=reaction)

sGlyPPP['name'] = name
stbg.model(sGlyPPP)

In [4]:
## Create stoichiometry
import GlyPPP_abg
S = st.stoich(GlyPPP_abg.model(),quiet=quiet)

## Display the extracted reactions

In [5]:
disp.Latex(st.sprintrl(sGlyPPP,chemformula=True))

<IPython.core.display.Latex object>

## Code to analyse pathways defined by chemostats and flowstats

In [6]:
## Analyse pathways defined by chemostats and flowstats
def ch(name):
    return '\\ch{'+name+'}'


def pathway(bg,phi,chemostats,flowstats=[],computePhi=False,verbose=False):
    """ Analyse pathways
    """
    
    print('Chemostats:',sorted(chemostats))
    print('Flowstats:', sorted(flowstats))
    ## Stoichiometry
    ## Create stoichiometry from bond graph.
    s = st.stoich(bg,quiet=True)

    ## Stoichiometry with chemostats
    sc = st.statify(s,chemostats=chemostats,flowstats=flowstats)

    ## Pathway stoichiometry
    sp = st.path(s,sc)
    
    ## Print info
    if verbose:
        for stat in sorted(chemostats):
            print(ch(stat)+',')

    ## Energetics
    if computePhi:
        Phi,Phip = energetics(s,sp,phi)
        #print('Phi units: kJ/mol')
#         fac = -F/1000
#         units='~\si{\kilo\joule\per\mol}'
        units = '~\si{\volt}'
        print(st.sprintp(sc))
        disp.Latex(st.sprintrl(sp,chemformula=True,Phi=Phip,showMu=showMu))
        #return s,sc,sp,Phi*fac,Phip*fac,units
        return s,sc,sp,Phip
    else:
        print(st.sprintrl(sp,chemformula=True))
        Phip = 0
        return s,sc,sp,Phip
  
def Pathway(S,chemostats,flowstats=[],verbose=False):
    """ Analyse pathways
    """
    
    print('Chemostats:',sorted(chemostats))
    print('Flowstats:', sorted(flowstats))
    ## Stoichiometry
    ## Create stoichiometry from bond graph.
    #s = st.stoich(bg,quiet=True)
    s = copy.copy(S)
    ## Stoichiometry with chemostats
    sc = st.statify(s,chemostats=chemostats,flowstats=flowstats)

    ## Pathway stoichiometry
    sp = st.path(s,sc)
    
    ## Print info
    if verbose:
        for stat in sorted(chemostats):
            print(ch(stat)+',')


    print(st.sprintrl(sp,chemformula=True))
    
    return s,sc,sp
    

# Analyse Pentose Phosphate Pathway with Glycolysis 
The pathways are isolated by using appropriate (zero-flow) flowstats. For compatibility with \citet[\S~18.2]{GarGri17} the pathways start from G6P (Glucose 6-phosphate).

## Common chemostats

In [7]:
def Chemostats(start='G6P',end=None):
    chemostats = ['ADP','ATP','CO2','H','H2O','NADP','NADPH']
    chemostats += [start]
    if end is not None:
        chemostats += end
    return chemostats           

## \ch{R5P} and \ch{NADPH} generation
- This pathway is isolated by setting PGI and TKT2 as flowstats and the
product \ch{R5P} is added to the chemostat list.

- It is isolated from the TCA cycle by replacing the connecting reactions (PDH and PFL) by flowstats. 

In [8]:
imp.reload(st)
print('R5P and NADPH generation')
chemostats = Chemostats(start='G6P',end=['R5P'])
flowstats = ['PGI','TKT2']
s,sc,sp = Pathway(S,chemostats,flowstats=flowstats)
disp.Latex(st.sprintrl(sp,chemformula=True))

<module 'stoich' from '/home/peterg/WEB/Github/Papers/GawPanCra21/stoich.py'>

R5P and NADPH generation
Chemostats: ['ADP', 'ATP', 'CO2', 'G6P', 'H', 'H2O', 'NADP', 'NADPH', 'R5P']
Flowstats: ['PGI', 'TKT2']
\begin{align}
\ch{G6P + H2O + 2 NADP &<>[ pr1 ] CO2 + 2 H + 2 NADPH + R5P }
\end{align}



<IPython.core.display.Latex object>

- The pathway reaction \ch{P_1} corresponds to the \ch{R5P} and
\ch{NADPH} synthesis discussed in comment 1 of 
<cite data-cite="GarGri17">Garrett and Grisham (2017)</cite>, p787.

- It is isolated from the TCA cycle by replacing the connecting reactions (PDH and PFL) by flowstats. 

## \ch{R5P} generation
- This pathway is isolated by setting GAPD and G6PDH2R as flowstats and the
product \ch{R5P} is added to the chemostat list.

In [9]:
print('R5P generation')
chemostats = Chemostats(start='G6P',end=['R5P'])
flowstats = ['G6PDH2R']
s,sc,sp = Pathway(S,chemostats,flowstats=flowstats)
disp.Latex(st.sprintrl(sp,chemformula=True))

R5P generation
Chemostats: ['ADP', 'ATP', 'CO2', 'G6P', 'H', 'H2O', 'NADP', 'NADPH', 'R5P']
Flowstats: ['G6PDH2R']
\begin{align}
\ch{ADP + H + 6 R5P &<>[ pr1 ] ATP + 5 G6P }
\end{align}



<IPython.core.display.Latex object>

## \ch{NADPH} generation

- This pathway is isolated by setting GAPD as a flowstat.

- It is isolated from the TCA cycle by replacing the connecting reactions (PDH and PFL) by flowstats. 

In [10]:
print('NADPH generation')
chemostats = Chemostats(start='G6P')
flowstats = []
s,sc,sp = Pathway(S,chemostats,flowstats=flowstats)
disp.Latex(st.sprintrl(sp,chemformula=True))

NADPH generation
Chemostats: ['ADP', 'ATP', 'CO2', 'G6P', 'H', 'H2O', 'NADP', 'NADPH']
Flowstats: []
\begin{align}
\ch{ADP + G6P + 6 H2O + 12 NADP &<>[ pr1 ] ATP + 6 CO2 + 11 H + 12 NADPH }
\end{align}



<IPython.core.display.Latex object>

- The pathway reaction \ch{pr1} corresponds to the 
\ch{NADPH} synthesis discussed in comment 3 of
<cite data-cite="GarGri17">Garrett and Grisham (2017)</cite>, p787.

